In [1]:
!ls ../../evaluations/hate_speech/task_b/

bert-combi.json			  beto_pred.json
bert-hier.json			  roberta-combi.json
bert.json			  roberta-hier.json
bertweet-combi.json		  roberta.json
bertweet-hier.json		  robertuito-combi.json
bertweet.json			  robertuito-hierarchical-gamma-0.1.json
beto-combi.json			  robertuito-hierarchical-gamma-0.json
beto-hierarchical-gamma-0.1.json  robertuito.json
beto-hierarchical-gamma-0.json	  robertuito_pred.json
beto.json


In [3]:
import json
import pandas as pd

evaluations = {

}

for model_name, path in [
    ('beto', '../../evaluations/hate_speech/task_b/beto.json'),
    ('beto-pred', '../../evaluations/hate_speech/task_b/beto_pred.json'),
    ('beto-hier', '../../evaluations/hate_speech/task_b/beto-hierarchical-gamma-0.1.json'),
    ('beto-combi', '../../evaluations/hate_speech/task_b/beto-combi.json'),
    ('robertuito', '../../evaluations/hate_speech/task_b/robertuito.json'),
    ('robertuito-hier', '../../evaluations/hate_speech/task_b/robertuito-hierarchical-gamma-0.1.json'),
    ('robertuito-combi', '../../evaluations/hate_speech/task_b/robertuito-combi.json'),
    ('bert', '../../evaluations/hate_speech/task_b/bert.json'),
    ('bert-hier', '../../evaluations/hate_speech/task_b/bert-hier.json'),
    ('bert-combi', '../../evaluations/hate_speech/task_b/bert-combi.json'),
    ('roberta', '../../evaluations/hate_speech/task_b/roberta.json'),
    ('roberta-hier', '../../evaluations/hate_speech/task_b/roberta-hier.json'),
    ('roberta-combi', '../../evaluations/hate_speech/task_b/roberta-combi.json'),
    ('bertweet', '../../evaluations/hate_speech/task_b/bertweet.json'),
    ('bertweet-hier', '../../evaluations/hate_speech/task_b/bertweet-hier.json'),
    ('bertweet-combi', '../../evaluations/hate_speech/task_b/bertweet-combi.json'),
    ]:
    with open(path) as f:
        evaluations[model_name] = json.load(f)


for key, evals in evaluations.items():
    print(f"We have {len(evals['evaluations']['hate_speech'])} {key} evaluations")



dfs = []
for model_name, model_results in evaluations.items():
    model_evaluations = model_results["evaluations"]["hate_speech"]
    
    if not model_evaluations:
        continue
    
    df = pd.DataFrame(model_evaluations)

    df.columns = [x.split("_", 1)[1] if "_" in x else x for x in df.columns]
    
    mean_df = pd.DataFrame({
        f"{model_name} mean": df.mean(), 
        f"{model_name} std": df.std()
    })
    dfs.append(mean_df)

result_df = pd.concat(dfs, axis=1)
index = [
    'hs_f1', 'tr_f1',
    'ag_f1', 'emr', 
    'macro_f1', 'macro_hs_f1_score',
]
pd.options.display.float_format = "{:.3f}".format
show_df = result_df.loc[index, [c for c in result_df.columns if "std" not in c]].T
#print(show_df.to_latex())

print(show_df.to_latex())

We have 10 beto evaluations
We have 10 beto-pred evaluations
We have 10 beto-hier evaluations
We have 10 beto-combi evaluations
We have 10 robertuito evaluations
We have 10 robertuito-hier evaluations
We have 10 robertuito-combi evaluations
We have 10 bert evaluations
We have 10 bert-hier evaluations
We have 10 bert-combi evaluations
We have 10 roberta evaluations
We have 10 roberta-hier evaluations
We have 10 roberta-combi evaluations
We have 10 bertweet evaluations
We have 10 bertweet-hier evaluations
We have 10 bertweet-combi evaluations
\begin{tabular}{lrrrrrr}
\toprule
{} &  hs\_f1 &  tr\_f1 &  ag\_f1 &   emr &  macro\_f1 &  macro\_hs\_f1\_score \\
\midrule
beto mean             &  0.741 &  0.765 &  0.688 & 0.685 &     0.731 &              0.771 \\
beto-pred mean        &  0.740 &  0.762 &  0.679 & 0.676 &     0.727 &              0.766 \\
beto-hier mean        &  0.735 &  0.758 &  0.674 & 0.703 &     0.722 &              0.776 \\
beto-combi mean       &  0.742 &  0.763 &  0.668 &

In [6]:
import scipy.stats

metric = "eval_emr"

combi = {
    "f1": [evaluation["eval_macro_f1"] for evaluation in evaluations["beto-combi"]["evaluations"]["hate_speech"]],
    "emr": [evaluation["eval_emr"] for evaluation in evaluations["beto-combi"]["evaluations"]["hate_speech"]]
}

hier = {
    "f1": [evaluation["eval_macro_f1"] for evaluation in evaluations["beto-hier"]["evaluations"]["hate_speech"]],
    "emr": [evaluation["eval_emr"] for evaluation in evaluations["beto-hier"]["evaluations"]["hate_speech"]]
}


standard = {
    "f1": [evaluation["test_macro_f1"] for evaluation in evaluations["beto"]["evaluations"]["hate_speech"]],
    "emr": [evaluation["test_emr"] for evaluation in evaluations["beto"]["evaluations"]["hate_speech"]]
}


Corremos un test de Kruskal para ver si la diferencia entre las medias es significativa


In [8]:
scipy.stats.kruskal(standard["emr"], hier["emr"], combi["emr"])

KruskalResult(statistic=0.6509018036072056, pvalue=0.7222016381935228)

In [96]:
scipy.stats.kruskal(standard["f1"], hier["f1"], combi["f1"])

KruskalResult(statistic=3.6567741935483866, pvalue=0.16067250810281022)

In [91]:
scipy.stats.mannwhitneyu(standard["emr"], hier["emr"], )

MannwhitneyuResult(statistic=26.5, pvalue=0.08186862373064437)

No lo es

In [44]:
import scipy.stats

combi_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito-combi"]["evaluations"]["hate_speech"]]
hier_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito-hier"]["evaluations"]["hate_speech"]]
standard_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito"]["evaluations"]["hate_speech"]]

scipy.stats.kruskal(standard_emr, hier_emr, combi_emr)

KruskalResult(statistic=10.860138146167559, pvalue=0.004382793059181017)

En el caso de robertuito sí! wtf